# gpt few-shot classifier

### Import

In [ ]:
import pandas as pd
from typing import List, Dict
from tqdm import tqdm
import time
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import os
import openai
import json
import uuid  
import json
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import csv



## Few-shot Examples

### Wep examples

In [ ]:
few_shot_examples = {
    "CONFESSION": [
        {"text": "הנאשם הורשע עפ\"י הודייתו, במסגרת הסדר טיעון בגדרו הוגש כתב אישום מתוקן, בעבירה שעניינה החזקת נשק ותחמושת שלא כדין, לפי סעיף 144(א) לחוק העונשין, תשל\"ז – 1977 (להלן: 'חוק העונשין').", "label": 1},
        {"text": "הסנגור עמד על נסיבות חייו הקשות של הנאשם, אובדנה של בתו לפני מספר שנים ובעיותיו הרפואיות, הודייתו בעבירה בהזדמנות הראשונה וחרטתו הכנה על מעשיו.", "label": 1},
        {"text": "הנאשם הודה בעובדות כתב אישום שעניינן, נשיאת והחזקת נשק שלא כדין.", "label": 1},
        {"text": "תחילה, כפר הנאשם במיוחס לו בכתב האישום.", "label": 1},
        {"text": "בדיון מיום 18/11/13 ולאחר שהוחל בשמיעת ראיות, הגיעו הצדדים להסדר טיעון ולפיו, הנאשם יחזור בו מכפירתו, כתב האישום יתוקן, הנאשם יודה בעובדות כתב האישום המתוקן ויורשע על פי הודאתו.", "label": 1}
    ],
    "CIR_TYPE_WEP": [
        {"text": "הנאשם הודה, כי נשא אקדח .", "label": 1},
        {"text": "הנאשם הורשע על סמך הודאתו בהסדר בכתב אישום מתוקן בהחזקת  רימון הלם.", "label": 1},
        {"text": "במסגרת העבירה נמכר תת מקלע מסוג קרלו.", "label": 1},
        {"text": "לאחר מספר דקות עצר עג\"אג\' את הרכב, השלושה יצאו ממנו ועג\"אג\' הוציא מתא המטען רובה סער מסוג M-4 ובכוחו להמית אדם (להלן: 'הרובה'), מחסנית תואמת ושקית ובה מספר רב של כדורים.", "label": 1},
        {"text": "כמו כן, הודה הנאשם, כי החזיק מתחת לכיסא הנהג שברכבו שקית ובה 17 כדורי אקדח ומחסנית לאקדח.", "label": 1},

    ],
    "CIR_HELD_WAY_WEP": [
        {"text": "הנאשם הודה, כי נשא אקדח הטעון במחסנית ריקה ובכיסו האחורי נמצאו 2 כדורים התואמים לאקדח.", "label": 1},
        {"text": "מתחת למושב הקדמי הימני נשא הנאשם מחסנית שבתוכה כדורים.", "label": 1},
        {"text": "באשר לסוגיית התכנון שקדם לביצוע העבירות והאם עסקינן במעשים אשר הינם פרי תכנון מוקדם, או מעשים רגעיים וספונטניים, נציין מחד, את עובדות כתב האישום המתוקן אשר מהן עולה, כי במועד אשר אינו ידוע למאשימה, הצטייד הנאשם בנשק מסוג אקדח וכי על רקע האירועים הקשורים לסכסוך, הגיע הנאשם למרכז הכפר כשהוא נושא ומוביל עמו את הנשק כשהוא טעון.", "label": 1},
        {"text": "על פי שירות המבחן, עת התייחס הנאשם לעבירה הנוכחית, הוא לקח אחריות על מעשיו והסביר כי נשא ברכבו אקדח, לא טעון וללא רישיון, נוכח תחושת איום ממשית לחייו ולחיי משפחתו, על רקע סכסוך עם אחרים שחשדו כי היה מעורב ברצח של מקורב אליהם.", "label": 1},
        {"text": "מאידך ראוי לציין, כי כעולה מעובדות כתב האישום, הרי שבפועל לא נעשה שימוש בנשק ולא נגרם כל נזק ממשי ומוחשי בעטיו, מבלי להקל ראש בעצם נשיאת הנשק, החזקת האקדח ברחובה של עיר כשהוא טעון ובנוכחות אחרים, עמם נתון הנאשם בסכסוך.", "label": 1}
    ],
    "CIR_AMMU_AMOUNT_WEP": [
        {"text": "הנאשם הודה, כי נשא אקדח הטעון במחסנית ריקה ובכיסו האחורי נמצאו 2 כדורים התואמים לאקדח.", "label": 1},
        {"text": "כמו כן, הודה הנאשם, כי החזיק מתחת לכיסא הנהג שברכבו שקית ובה 17 כדורי אקדח ומחסנית .", "label": 1},
        {"text": "מתחת למושב הקדמי הימני נשא הנאשם את האקדח ובו מחסנית ריקה.", "label": 1},
        {"text": "זאת עשה הנאשם בלא רשות על פי דין לנשיאתם והחזקתם של האקדח והמחסנית.", "label": 1},
        {"text": "כמו כן, הודה הנאשם, כי החזיק מתחת לכיסא הנהג שברכבו שקית ובה 17 כדורי אקדח ומחסנית לאקדח.", "label": 1}
    ],
    "CIR_PURPOSE": [
        {"text": "בעת הצגת ההסדר ציינו ב\"כ הצדדים, כי אין מחלוקת שהנאשם החזיק בנשק לצורך הגנה עצמית.", "label": 1},
        {"text": "בהקשר זה ראוי לציין, כי הנאשם החזיק שלא כדין אקדח והגיע עמו לטבורו של כפר, כשהוא טעון, זאת במטרה להפחיד אחרים המעורבים בסכסוך ואשר היו בקרבת מקום.", "label": 1},
        {"text": "באשר לנזק שצפוי היה להיגרם מביצוע העבירות, יצוין בהקשר זה האמור בעובדות כתב האישום המתוקן ולפיהן, הנאשם הגיע עם הנשק כשהוא טעון, במטרה להפחיד את המעורבים בסכסוך אשר היו בקרבת מקום, על המשמעויות הרוחביות הקמות מכך, באשר לנזק שעלול היה להיגרם מהחזקת אקדח כשהוא טעון ברחובה של עיר, בנוכחות אחרים, אשר מעורבים בסכסוך עם הנאשם ואחרים.", "label": 1},
        {"text": "כמו כן, לשיטתה העובדה כי האקדח נמצא טעון במחסנית מעידה על כך כי לא הוחזק ל\"מטרות שלום\", כדבריה.", "label": 1},
        {"text": "הנאשם הטעים כי תחושות הסכנה והפחד התגברו לאחר הפגיעה בבנו, או אז, גמלה בלבו ההחלטה להשיג כלי נשק, למען הגנתו.", "label": 1}
    ],
    "GENERAL_CIRCUM": [
        {"text": "הנאשם שייך גם את מעורבותו בעבירות דנן וגם את אורח החיים שניהל לגילו הצעיר, לנטייתו לחפש אחר חופש, שייכות בקרב קבוצת השווים וחוסר התחשבות בגבולות.", "label": 1},
        {"text": "שירות המבחן ציין כי ההליך הפלילי היווה גורם מרתיע לנאשם.", "label": 1},
        {"text": "הסנגור עמד על נסיבות חייו הקשות של הנאשם, אובדנה של בתו לפני מספר שנים ובעיותיו הרפואיות, הודייתו בעבירה בהזדמנות הראשונה וחרטתו הכנה על מעשיו.", "label": 1},
        {"text": "זאת ועוד, הנאשם נמנע מחיי חברה שוליים וכפי העולה מתסקיר שירות המבחן, הוא ביטא כוחות תוך שמירה על גבולות, הודה במיוחס לו, לקח אחריות על המעשים והביע חרטה מלאה וכנה בשלהם.", "label": 1},
        {"text": "בנסיבות אלו, נוכח מקבץ הנסיבות אשר אינן קשורות בביצוע העבירות ובכלל זה, האמור בליבת תסקירי שירות המבחן שהוגשו בעניינו של הנאשם, האמור בתסקיר הראשון וההמלצה המובאת בסופו, מנגד, השוני באמור בתסקירים המשלימים שהוגשו וההמלצה העדכנית המובאת בתסקיר המסכם שהוגש, כמו גם הודאת הנאשם, לקיחת האחריות, חיסכון זמן שיפוטי יקר, הצער והחרטה שהביע הנאשם בשל המעשים, נסיבותיו האישיות והמשפחתיות של הנאשם, העובדה, כי עסקינן בנאשם צעיר נעדר כל הרשעות קודמות ואשר זוהי לו עשייתו הראשונה בפלילים, באתי לכלל מסקנה, כי יש למקם את העונש הראוי לנאשם בתוך מתחם העונש ההולם שקבענו לעיל, ברף הנמוך של מתחם העונש ההולם אך לא בצד הקיצון שבו.", "label": 1}
    ],
    "CIR_STATUS_WEP": [
        {"text": "  בנוסף על פי העבירה ניתן לומר שהנאשם ביצע ירי באזור מגורים.", "label": 1},
        {"text": "הנאשם הודה, כי נשא אקדח הטעון במחסנית ריקה ובכיסו האחורי נמצאו 2 כדורים התואמים לאקדח.", "label": 1},
        {"text": "מדובר בכלי נשק שמסוגל לירות כדור שבכוחו להמית אדם", "label": 0},
        {"text": "בנוסף, יש לקחת בחשבון, את העובדה כי הנשק אינו תקין.", "label": 1},
        {"text": "יצוין, כי דבריו אלו של הנאשם, עומדים הם במידה מסוימת בסתירה לעובדות כתב האישום, שעה שמהעובדות עולה כאמור, כי הנאשם הצטייד באקדח והגיע למרכז הכפר כשהוא נושא ומוביל עמו את הנשק כשהוא טעון.", "label": 1}
    ],
    "REGRET": [
        {"text": "הסנגור עמד על נסיבות חייו הקשות של הנאשם, אובדנה של בתו לפני מספר שנים ובעיותיו הרפואיות, הודייתו בעבירה בהזדמנות הראשונה וחרטתו הכנה על מעשיו.", "label": 1},
        {"text": "זאת ועוד, הנאשם נמנע מחיי חברה שוליים וכפי העולה מתסקיר שירות המבחן, הוא ביטא כוחות תוך שמירה על גבולות, הודה במיוחס לו, לקח אחריות על המעשים והביע חרטה מלאה וכנה בשלהם.", "label": 1},
        {"text": "בנסיבות אלו, נוכח מקבץ הנסיבות אשר אינן קשורות בביצוע העבירות ובכלל זה, האמור בליבת תסקירי שירות המבחן שהוגשו בעניינו של הנאשם, האמור בתסקיר הראשון וההמלצה המובאת בסופו, מנגד, השוני באמור בתסקירים המשלימים שהוגשו וההמלצה העדכנית המובאת בתסקיר המסכם שהוגש, כמו גם הודאת הנאשם, לקיחת האחריות, חיסכון זמן שיפוטי יקר, הצער והחרטה שהביע הנאשם בשל המעשים, נסיבותיו האישיות והמשפחתיות של הנאשם, העובדה, כי עסקינן בנאשם צעיר נעדר כל הרשעות קודמות ואשר זוהי לו עשייתו הראשונה בפלילים, באתי לכלל מסקנה, כי יש למקם את העונש הראוי לנאשם בתוך מתחם העונש ההולם שקבענו לעיל, ברף הנמוך של מתחם העונש ההולם אך לא בצד הקיצון שבו.", "label": 1},
        {"text": "מנגד, אף עמדתה העונשית של המאשימה, חורגת היא לחומרא, אינה עולה בקנה אחד עם מתחם העונש ההולם שקבענו ועם מיקום העונש הראוי לנאשם ואינה משקללת נכונה את נתוני העושה, נסיבותיו האישיות, היות הנאשם נעדר עבר פלילי, גילו הצעיר, הודאת הנאשם, הצער והחרטה שהביע בשל המעשים, כמו גם ההליך הטיפולי בו החל ליטול חלק והמלצת שירות המבחן כפי תסקירו האחרון.", "label": 1},
        {"text": "הסנגור ציין כי ההגנה אינה מקלה ראש בעבירה דנן, ומודעת לסכנה הטמונה בעבירות נשק, אך ביקש להתחשב לקולא בגזירת עונשו בנסיבות ביצוע העבירה, בנסיבותיו האישיות של הנאשם, בהודייתו בביצוע העבירה ובחרטתו הכנה והאמיתית.", "label": 1}
    ],
"PUNISHMENT": [
    {"text": "12 חודשי מאסר על תנאי והתנאי הוא שהנאשם לא יעבור תוך תקופה של 3 שנים כל עבירה בנשק מסוג פשע ויורשע בגינה.", "label": 1},
    {"text": "12 חודשי מאסר על תנאי למשך 3 שנים והתנאי הוא שהנאשם לא יעבור עבירה בנשק.", "label": 1},
    {"text": "א.\t9 חודשי מאסר בפועל בניכוי ימי מעצרו מיום 8/4/13 ועד ליום 11/7/13.", "label": 1},
    {"text": "עבודות במסגרת של\\\"צ בהיקף של 500 שעות, עפ\\\"י תכנית שתוכן ע\\\"י שירות המבחן ותומצא בכתב לבית המשפט.", "label": 1},
    {"text": "6 חודשי מאסר על תנאי והתנאי הוא שהנאשם לא יעבור תוך תקופה של 3 שנים כל עבירה בנשק מסוג עוון ויורשע בגינה.", "label": 1}
],
    "CIR_PLANNING": [
        {"text": "באשר לסוגיית התכנון שקדם לביצוע העבירות והאם עסקינן במעשים אשר הינם פרי תכנון מוקדם, או מעשים רגעיים וספונטניים, נציין מחד, את עובדות כתב האישום המתוקן אשר מהן עולה, כי במועד אשר אינו ידוע למאשימה, הצטייד הנאשם בנשק מסוג אקדח וכי על רקע האירועים הקשורים לסכסוך, הגיע הנאשם למרכז הכפר כשהוא נושא ומוביל עמו את הנשק כשהוא טעון.", "label": 1},
        {"text": "מעשה כזה, כך לטענת המאשימה, הינו מעשה מתוכן וכי הנאשם אף מודה, כי הייתה בהחזקת הנשק הטעון כוונה להפחיד ואף ציין כיצד התכוון להפחיד את הזולת כשהנשק טעון.", "label": 1},
        {"text": "הנאשם תכנן את מעשיו מבעוד מועד.", "label": 1},
        {"text": "נטען שאין מדובר בטעות בשיקול דעת או במעידה רגעית כאשר הנאשם הגיע במודע הן למקום העבודה והן לאחר מכן לבית המתלונן כאשר הוא מצוייד בנשק בו הוא עשה שימוש.", "label": 1},
        {"text": "לגבי נסיבות ביצוע העבירה, ציינה כי מדובר במעשה שבוצע לאחר תכנון מוקדם, תכנון מתמשך לאחר התחברות לאחר.", "label": 1},
  ]  ,
    "RESPO": [
        {"text": "באשר לעבירות דנן, הרי שהנאשם התקשה לקבל אחריות לעובדות כתב האישום המתוקן.", "label": 1},
        {"text": "הודיה זו לא זו בלבד שהובילה לחסכון בזמן שיפוטי יקר, אלא גם מגלמת בתוכה נטילת אחריות על מעשיו.", "label": 1},
        {"text": "שירות המבחן לא נתן אמון מלא בגרסתו והעריך שנטילת האחריות שלו היא חלקית בלבד.", "label": 1},
        {"text": "הנאשם הביע חרטה על מעשיו ולקח עליהם אחריות.", "label": 1},
        {"text": "זאת ועוד, הנאשם לקח אחריות על המעשים .", "label": 1},


    ],
        "CIR_USE": [
        {"text": "הגיע הנאשם למקום, ויידה שלושה בקבוקי תבערה, ממרחק של 20 מטר, לעבר הרכבים של כוחות הביטחון.", "label": 1},
        {"text": "הנאשם הורשע בעבירה של ירי באזור מגורים.", "label": 1},
        {"text": "הנאשם ניסה לירות בנשק המאולתר לעבר רגליו של אחד מהאחרים, אך הנשק לא פעל.", "label": 1},
        {"text": "במועד האירוע הושלך הרימון לעבר מרפסת ביתו של המתלונן, לאחר שהנצרה שוחררה.", "label": 1},
        {"text": "מעבר להחזקתו בתת המקלע המאולתר, הודה הנאשם כי במהלך שנת 2011 הובילו ונשאו למקום פתוח, שם ירה באמצעותו כ-20 כדורים.", "label": 1},
    ],
    "CIR_OBTAIN_WAY_WEP": [
        {"text": "מנגד, נציין את דברי הנאשם בפני שירות המבחן, כפי שהדבר קיבל את ביטויו בתסקירי שירות המבחן שהוגשו, בדבר קבלת האקדח במקום האירוע במרכז הכפר, ממאן דהוא, אשר הנאשם אינו זוכר את זהותו ושעה שהנאשם היה נתון תחת השפעת אלכוהול.", "label": 1},
        {"text": "הנאשם הכחיש, כי הוא הגיע לקטטה כשהוא מצויד באקדח ולטענתו, האקדח ניתן לו בעת האירוע מאדם שאינו זוכר, בשל היותו תחת השפעת אלכוהול.", "label": 1},
        {"text": "באישום השלישי החזיק הנאשם אקדח שקיבל מאביו, בביתו של בן דודו בכפר עקב, וביום סיום הבגרויות לקח את האקדח וירה שמונה כדורים באוויר.", "label": 1},
        {"text": "לאחר מכן קיבל הנאשם מאביו אקדח בלגי, אותו החזיק בבית של בן דודו.", "label": 1},
        {"text": "במועד כלשהו שוחח הנאשם עם סאמי וזאת כדי לרכוש ממנו נשק.", "label": 0},
        {"text": "באשר לעבירה, הנאשם ייצר את המטען.", "label": 1}
    ]
}

### Drug examples

In [ ]:
few_shot_examples_drug = {
    "CIR_TYPE": [
        {"text": "בהמשך נפגשו השניים בפארק, אז מכר לו הנאשם גרם קנבוס בתמורה ל-120 ₪.", "label": 1},
        {"text": "ביום 20.10.22 שב הנאשם לישראל כשהוא נושא עמו כ-25,303 טבליות סם מסוכן מסוג MDMA שהוטמנו בתא ייעודי שבנה בתחתית מזוודתו, אותו קיבע באמצעות קרש עץ.", "label": 1},
        {"text": "עמית הסכים, אך ביקש כי במקום תשלום של 5,000 ₪ ייתן לו הנאשם סם מסוכן מסוג קטמין בשווי של 5,000 ₪ הנאשם הסכים ואמר לו כשיגיעו לארץ, יתחשבנו.", "label": 1},
        {"text": "כעבור מספר דקות, הגיע גם הנאשם למקום על אופניו, ניגש לעבר חלון רכבו של הסוכן ומסר לו גוש עטוף בנייר ובו 99.53 גרם נטו של סם מסוכן מסוג קוקאין.", "label": 1},
        {"text": "לפי המתואר בכתב האישום, ביום 14.04.23 גמלה בליבו של הנאשם החלטה לייבא לארץ סם מסוכן מסוג GHB.", "label": 1},

    ],
    "CIR_AMOUNT": [
        {"text": "ביום 20.10.22 שב הנאשם לישראל כשהוא נושא עמו כ-25,303 טבליות סם מסוכן מסוג MDMA שהוטמנו בתא ייעודי שבנה בתחתית מזוודתו, אותו קיבע באמצעות קרש עץ.", "label": 1},
        {"text": "הנאשם פתח את החבילה והוציא ממנה את שני הכדים שהכילו כ- 350 גרם נטו של סם מסוכן מסוג KETAMINE מסר אותם לידי עמית ואמר לו לשים אותם במושב הנהג בהונדה.", "label": 1},
        {"text": "בתאריך 14.8.22 בשעה 14:02, הגיע הסוכן למקום ושם פגש את הנאשם שהעביר לו עטיפת נייר ובה שקית המכילה גוש של סם מסוכן מסוג קוקאין במשקל 119.89 גרם נטו.", "label": 1},
        {"text": 'במעבדה נתפסו 137.6 ק"ג של סם מסוכן מסוג קנביס;', "label": 1},
        {"text": 'בתיק המצורף החזיק הנאשם, לצריכתו העצמית, בסם מסוכן מסוג קנביס במשקל של 98.6 גרם.', "label": 1}
    ],
    "GENERAL_CIRCUM": [
        {"text": "הנאשם שייך גם את מעורבותו בעבירות דנן וגם את אורח החיים שניהל לגילו הצעיר, לנטייתו לחפש אחר חופש, שייכות בקרב קבוצת השווים וחוסר התחשבות בגבולות.", "label": 1},
        {"text": "עוד עולה, כי עד להסתבכותו של הנאשם במקרה דנן, ניהל הנאשם אורח חיים פרודוקטיבי ונורמטיבי ועשה לביתו ולפרנסתו.", "label": 1},
        {"text": "הסנגור עמד על נסיבות חייו הקשות של הנאשם, אובדנה של בתו לפני מספר שנים ובעיותיו הרפואיות, הודייתו בעבירה בהזדמנות הראשונה וחרטתו הכנה על מעשיו.", "label": 1},
        {"text": "זאת ועוד, הנאשם נמנע מחיי חברה שוליים וכפי העולה מתסקיר שירות המבחן, הוא ביטא כוחות תוך שמירה על גבולות, הודה במיוחס לו, לקח אחריות על המעשים והביע חרטה מלאה וכנה בשלהם.", "label": 1},
        {"text": "בנסיבות אלו, נוכח מקבץ הנסיבות אשר אינן קשורות בביצוע העבירות ובכלל זה, האמור בליבת תסקירי שירות המבחן שהוגשו בעניינו של הנאשם, האמור בתסקיר הראשון וההמלצה המובאת בסופו, מנגד, השוני באמור בתסקירים המשלימים שהוגשו וההמלצה העדכנית המובאת בתסקיר המסכם שהוגש, כמו גם הודאת הנאשם, לקיחת האחריות, חיסכון זמן שיפוטי יקר, הצער והחרטה שהביע הנאשם בשל המעשים, נסיבותיו האישיות והמשפחתיות של הנאשם, העובדה, כי עסקינן בנאשם צעיר נעדר כל הרשעות קודמות ואשר זוהי לו עשייתו הראשונה בפלילים, באתי לכלל מסקנה, כי יש למקם את העונש הראוי לנאשם בתוך מתחם העונש ההולם שקבענו לעיל, ברף הנמוך של מתחם העונש ההולם אך לא בצד הקיצון שבו.", "label": 1}
    ],
"PUNISHMENT": [
    {"text": "12 חודשי מאסר על תנאי והתנאי הוא שהנאשם לא יעבור תוך תקופה של 3 שנים כל עבירה בנשק מסוג פשע ויורשע בגינה.", "label": 1},
    {"text": "12 חודשי מאסר על תנאי למשך 3 שנים והתנאי הוא שהנאשם לא יעבור עבירה בנשק.", "label": 1},
    {"text": "א.\t9 חודשי מאסר בפועל בניכוי ימי מעצרו מיום 8/4/13 ועד ליום 11/7/13.", "label": 1},
    {"text": "עבודות במסגרת של\\\"צ בהיקף של 500 שעות, עפ\\\"י תכנית שתוכן ע\\\"י שירות המבחן ותומצא בכתב לבית המשפט.", "label": 1},
    {"text": "6 חודשי מאסר על תנאי והתנאי הוא שהנאשם לא יעבור תוך תקופה של 3 שנים כל עבירה בנשק מסוג עוון ויורשע בגינה.", "label": 1}
],
        "CIR_ROLE": [
        {"text": "הנאשם הינו הגורם המרכזי, המתכנן והמוציא אל הפועל של העבירה, תוך הסתייעות באחר והפעלתו.", "label": 1},
        {"text": "הנאשם ייבא סם מסוכן תוך תכנון, הסלקה והפעלת גורמים נוספים, כאשר מדובר בסם מסוכן בכמות גדולה, סם אשר כאמור בת.פ.", "label": 1},
        {"text": "באותן הנסיבות שימש המתלונן כשליח של השניים לצורך ביצוע עסקאות סם.", "label": 1},
        {"text": "ובמקרה דנן, מעמדו של הנאשם כמתווך נמצא גבוה בהיררכיה של שרשרת הפצת הסם.", "label": 1},
        {"text": "הנאשם ביצע את כל העסקאות באופן עצמאי וללא כל עזרה של גורם מתווך כלשהו.", "label": 1}
    ],
    "CIR_EQ": [
        {"text": "בנוסף לאמור, נתפס בוילה ציוד רב ששימש לגידול הסם.", "label": 1},
        {"text": "לשם כך התקין בה ציוד להפקה, הכנה וטיפול בקנבוס, כולל גופי תאורה ייעודיים, מערכות אוורור והשקיה, אדניות וחומרי דישון.", "label": 1},
        {"text": 'ב"כ המאשימה טענה לעניין כתב האישום כי מדובר בתעוזה, מעבדה שהנאשם הקים בדירת מגוריו, שם ילדיו גרים, שם מקיים חיי משפחה, ומצייד אותה בציוד יקר ערך.', "label": 1},
        {"text": "לטענתו, אין המדובר במעבדה קטנה, זניחה ומאולתרת, אלא במעבדת סמים משוכללת.", "label": 1},
        {"text": "בענייננו, הקים הנאשם מעבדת סמים במחסן בחצר ביתו ורכש ציוד ייעודי וכלים שונים וביניהם מד טמפרטורה, מאווררים, צינורות השקיה, מנורות חימום, דישון ועוד – הכל לצורך גידול הסם והכנתו.", "label": 1}
    ],
        "REGRET": [
        {"text": "בהודאתו, חסך הנאשם זמן שיפוטי והביע חרטה.", "label": 1},
        {"text": "שירות המבחן ציין, כי הנאשם לקח אחריות מלאה למעשיו ומצר על ביצוע העבירה, ומודע כיום לנזקים וההשלכות הכרוכים בביצועה.", "label": 1},
        {"text": "בהתייחסותו לעבירה הנוכחית הוא לקח אחריות, ביטא צער וחרטה בגינם לצד הכרה בחומרתם.", "label": 1},
        {"text": "בדבריו לבית המשפט הנאשם הביע צער על מעשיו, התנצל, אמר שהוא מעוניין לגדל את בנו, לחיות איתו, להיות אבא טוב ולעבור שינוי.", "label": 1},
        {"text": "עוד התרשם שירות המבחן, כי הנאשם מבטא מודעות למצבו, מביע חרטה כנה על האופן שבו פעל, ואף הביא דוגמאות מחייו כיום המעידות על שינוי דפוסים.", "label": 1}
    ],
    "RESPO": [
        {"text": "כ הנאשם ציין: המדובר בנאשם צעיר שלקח אחריות, הודה וחסך זמן שיפוטי ניכר.", "label": 1},
        {"text": "מידת הפגיעה בערך המוגן; גילו של הנאשם; הנסיבות עליהן עמדו הצדדים בטיעוניהם; הודאתו המצביעה על נטילת אחריות.", "label": 1},
        {"text": "הנאשם עצמו הצטער על מעשיו, קיבל אחריות עליהם, וציין כי הוא חש בושה על ביצוע העבירות.", "label": 1},
        {"text": "מתסקיר שירות המבחן עולה שהנאשם עבר תהליך של חקירה עצמית עמוקה ושינוי מחשבתי, מקפיד לשמור על החוק והסדר החברתי ומבטא נכונות להשתלב בטיפול.", "label": 1},
        {"text": "לקולה אני מביא בחשבון את: קבלת האחריות; הבעת החרטה; ההודאה בכתב האישום המתוקן, החיסכון בזמן שיפוטי רב.", "label": 1}
    ],
    "CONFESSION": [
        {"text": "ביום 27.11.22 הגיעו הצדדים להסדר לפיו, הנאשם הודה והורשע בעובדות כתב האישום.", "label": 1},
        {"text": "לזכותו של הנאשם המאשימה זקפה את הודאתו.", "label": 1},
        {"text": "עוד הדגיש את התנהלותו לאחר תפיסתו, לרבות הודאתו בשלב מוקדם, הירתמותו הטוטאלית להליך טיפולי אינטנסיבי בקהילה לגמילה מסמים.", "label": 1},
        {"text": "לא מצאתי לחרוג ממתחם העונש ההולם אותו קבעתי ובגדרי המתחם נתתי משקל להודאתו של הנאשם ולחיסכון בזמן שיפוטי.", "label": 1},
        {"text": "עם זאת, יש בשיקולים לקולא, ובעיקר ההודיה, חלקו של הנאשם בשרשרת, נסיבות גיוסו והעובדה כי זהו מאסרו הראשון.", "label": 1}
    ]
}

## Classifiers

### Wep Classifier

In [ ]:

class HebrewLegalMultiLabelClassifierWep:
    """
    A Hebrew legal text classifier using OpenAI's GPT models.
    This classifier supports zero-shot, few-shot, and chain-of-thought prompting techniques
    to determine whether a given legal text contains specific predefined labels.
    """

    def __init__(self, output_path: str, model_name: str = "gpt-4o-mini"):
        """
        Initializes the classifier with the specified model and output path.
        
        Args:
            output_path (str): Path to save classification results.
            model_name (str, optional): OpenAI model name. Defaults to "gpt-4o-mini".
        """

        self.client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        self.model_name = model_name
        self.output_path = output_path
        self.label_descriptions = {
            'CONFESSION': "האם מופיעה הודאה או הכחשה מפורשת של ביצוע העבירה? תייג נכון אם מופיע ביטוי שמעיד על כך שהנאשם הודה או כפר במעשיו לדוגמא : הודייתו, הודאה,כפירתו וכו...",
            'CIR_TYPE_WEP': 'האם מופיע סוג הנשק המעורב בעבירה? יש לתייג נכון אם הטקסט מציין מפורשות סוג ספציפי של נשק, כמו: רובים ואקדחים (רובה סער, תת-מקלע, אקדח, קלצניקוב, וכו), חומרי נפץ (מטען חבלה, רימון, בקבוק תבערה, מטען צינור וכו). אין לתייג נכון אם: הטקסט מתאר שימוש בנשק מבלי לציין סוג ספציפי,הטקסט משתמש במונחים כלליים כמו נשק חם, נשק התקפי, נשק אוטומטי ללא פירוט נוסף',
            'CIR_HELD_WAY_WEP': 'האם מופיע תיאור ברור לאופן החזקת הנשק? יש לתייג "נכון" אם הטקסט מתאר היכן הנשק עצמו הוחזק (למשל: בבית, ברכב, על גופו, מוסתר במקום מסוים) או כיצד הוסתר/נשא אותו. ניתן לכלול גם אביזרים נלווים (מחסנית, תחמושת וכו) בתנאי שהתיאור כולל גם את הנשק עצמו. אין לתייג "נכון" אם טקסט מתאר רק את עצם ההחזקה של הנשק בלי מידע על מיקום או דרך אחסון. אין לתייג "נכון" אם יש תיאור של אביזרים בלבד, ללא ציון של הנשק עצמו.',
            'CIR_AMMU_AMOUNT_WEP': "האם מופיעים נתונים כמותיים על התחמושת (כמו מספר כדורים,קליעים, מחסניות)? אם מופיע תיאור של מחסנית בלבד ללא פירוט על מספר כדורים, יש להניח שמדובר במחסנית מלאה.יש לתייג נכון גם במקרים בהם הנשק עצמו הוא גם התחמושת(לדוגמא: פגזים),",
            'CIR_PURPOSE' : """
            האם מופיעה כוונה או מטרה לביצוע העבירה?
            תייג 'נכון' אם הטקסט מציין במפורש או במשתמע בבירור את הסיבה לביצוע העבירה, למשל בצע כסף, הגנה עצמית, סכסוך, תדמית, פיגוע ביטחוני וכו'.
            תייג 'נכון' גם אם הכוונה מצוינת כהיעדר כוונה או מטרה ספציפית (למשל, עבירה שבוצעה מתוך רשלנות או בטעות).
            אין לתייג 'נכון' במקרים הבאים:
            - אם הטקסט מתאר רק את המעשה בלי פירוט הסיבה.
            - אם מדובר רק על חומרת העבירה, גזר הדין או השלכותיה.
            - אם יש דיון בשיקום או נסיבות חיי הנאשם ללא קשר למניע.
            - אם ההקשר הוא משפטי בלבד ולא קשור ישירות למניע.
            """,
            'GENERAL_CIRCUM': (
                "האם מופיע נסיבות כלליות להקלה על העונש?"
                "יש לתייג נכון גם אם מופיע אחד מהקריטריונים הבאים באופן משתמע, כל עוד ברור מההקשר שהם מתקיימים :"
                "(1) הפגיעה האפשרית של העונש בנאשם כולל פגיעה בלתי מידתית בנאשם בשל מאסר ארוך או נסיבותיו האישיות. "
                "(2) הפגיעה האפשרית של העונש במשפחתו של הנאשם; "
                "(3) הנזקים שנגרמו לנאשם מביצוע העבירה ומהרשעתו; "
                "(4) מאמצי הנאשם לשיקום ,שיקום עצמי, חזרה לחיים נורמטיביים , התרשמות שירות המבחן מהליך שיקום, שינוי חיובי,הרתעת ההליך הפלילי לנאשם"
                "(5) ניסיון הנאשם לתיקון תוצאות העבירה ולפיצוי הנפגעים; "
                "(6) שיתוף פעולה עם רשויות החוק, כולל אם ההליך הפלילי השפיע על הנאשם באופן מרתיע או משקם. "
                "(7) התנהגות חיובית של הנאשם ותרומתו לחברה "
                "(8)  נסיבות חייו של הנאשם טרם ביצוע העבירה, כולל השפעה של גיל, פחד, לחץ חברתי או רגשי על המעשים , מצוקה כלכלית או פגיעה אפשרית בפרנסה. בעיות רפואיות, נפשיות, או חוויות טראומטיות שהובילו לעבירה. השפעת החברה הסובבת, כולל התמכרויות ,הדרדרות לסמים, התדרדרות סביבתית. "
                "(9) התנהלות חריגה מצד רשויות החוק (עיכובים, עוולות וכו'). "
                "(10) חלוף זמן משמעותי מאז העבירה והשינוי שחל בנאשם מאז."
                "(11)  עברו הפלילי של הנאשם, כולל הרשעות קודמות"
                "אין לתייג נכון :אזכורים כלליים של חוקי ענישה,סיבות טכניות בלבד כמו חיסכון בזמן שיפוטי."
                "אין לתייג נכון אם מוזכר רק המלצת שירות המבחן ללא פירוט נוסף"
            ),
            'CIR_STATUS_WEP': " האם מופיע תיאור של מצב הנשק? תייג 'נכון' אם הטקסט מתאר במפורש את מצב הנשק, כולל: - נשק תקול (לא פועל, מקולקל, לא ירה). - נשק מפורק (מורכב מחלקים, הוסרו ממנו רכיבים). - נשק מופרד מתחמושת (כאשר נשק ותחמושת מוזכרים יחד אך אין אינדיקציה שהם מחוברים). - נשק עם מחסנית בהכנס (מחסנית בתוך הנשק, אך לא בהכרח טעון). - נשק עם כדור בקנה (טעון ומוכן לירי). - נשק תקין (דרוך, פעיל, או מוכן לשימוש).  אם הטקסט מכיל התייחסות לנשק ולתחמושת, יש להניח כברירת מחדל שהם מופרדים, אלא אם נאמר אחרת. אין לתייג נכון אם מדובר במצב התחמושת בלבד ללא הנשק.",
            'REGRET': "האם מופיעים ביטויים המעידים על חרטה או חוסר חרטה של הנאשם? תייג נכון אם מופיע ביטוי שמעיד על כך שהנאשם התחרט או לא התחרט על מעשיו לדוגמא : הביע חרטה, חרטתו ,לא מתחרט וכו",
            'PUNISHMENT': "האם מופיע פירוט מדויק של העונש שהוטל בהחלטה הנוכחית בלבד?  תייג 'נכון' רק אם מדובר בעונש שניתן במסגרת פסק הדין הנוכחי ולא עונש שנגזר בתיקים קודמים.  פירוט העונש יכול לכלול:  - מאסר בפועל, כולל חופף/מצטבר  - מאסר על תנאי  - חילוט רכוש  - קנסות ופיצויים  - עבודות שירות  - כל עונש נוסף שנפסק במסגרת ההחלטה הנוכחית בלבד.  אם העונש מתייחס להרשעות קודמות, אין לתייג 'נכון'.",
            'CIR_PLANNING': "האם מופיעים סימנים המעידים על תכנון מוקדם של העבירה או חוסר בתכנון מוקדם? תייג 'נכון' אם הטקסט כולל אזכור מפורש של תכנון, בין אם הטקסט טוען שהיה תכנון או אם יש הכחשה ישירה של תכנון, צעדים טכניים של הכנה או פעולות שאינן כוללות ראיה ברורה לכך שהנאשם תכנן מראש לבצע עבירה – אינן נחשבות תכנון.",
            'RESPO':'האם מופיעים ביטויים המעידים על קבלת או אי לקיחת אחריות לביצוע העבירה? תייג נכון אם מופיע ביטוי שמעיד על כך שהנאשם לוקח בצורה מלאה או חלקית או לא לוקח אחריות על מעשיו. אין לתייג נכון אם מופיע בטקסט רק הבעת חרטה, צער או הכרה בטעות  ',
            'CIR_OBTAIN_WAY_WEP':'.האם מופיע תיאור של הדרך בה הנשק הגיע לידי הנאשם, במפורש או במשתמע? יש לתייג נכון רק אם ניתן להבין כיצד הנשק הגיע אליו מלכתחילה ,אין לתייג נכון אם מדובר רק בהחזקת נשק או בהעברה שלו ממקום למקום ללא הסבר על אופן ההשגה הראשוני- רכישה, ייצור, גניבה וכו׳.... אם מדובר בעסקת נשק, יש לתייג נכון רק אם מצוין שהעסקה הושלמה. אין לתייג נכון אם הטקסט מתאר רק תכנון, ניסיון או הצעה לרכוש נשק, בלי אישור שהנשק התקבל בפועל',
            'CIR_USE':'האם מופיע תיאור של שימוש פעיל באמצעי לחימה או חוסר שימוש באמצעי לחימה על ידי הנאשם במהלך העבירה? שימוש פעיל כולל ירי, הפעלת מטען חבלה, הצתת פתיל,השלכת רימון או כל אקט שמטרתו להפעיל אמצעי לחימה . החזקה, נשיאה או טעינת נשק ללא הפעלה אינם נחשבים כשימוש פעיל.'

        }
        self.output_path = output_path
        
    def create_zero_shot_prompt(self, text: str, label: str) -> str:
        """
        Generates a zero-shot prompt for classification.
        
        Args:
            text (str): The input legal text.
            label (str): The label to classify.
        
        Returns:
            str: The formatted prompt.
        """

        label_description = self.label_descriptions.get(label, None)
    
        if label_description is None:
            # If label description is not found, skip this label and continue
            print(f"Label '{label}' not found in label_descriptions. Skipping...")
            return None  # Or return a default prompt if needed, depending on your logic

        prompt = f"""קבע אם הטקסט המשפטי כולל את המידע הבא: {label_description}.
        התשובה יכולה להיות רק 'נכון' או 'לא נכון'.
        ענה אך ורק 'נכון' או 'לא נכון' ללא הסברים נוספים.
        
    הנחות:
    1. אם לא מצוין במפורש אדם אחר, יש להניח שההתייחסות היא לנאשם.
    2. אם הטקסט מתייחס לתיק אחר (למשל, "במקרה של ע"פ 11/22 הנאשם ירה..."), אין לתייג אותו כ'נכון' באף מקרה.
    3. אם מצוין שם של אדם אחר שאינו הנאשם (למשל, "פלוני ירה..."), אין לתייג את הטקסט כ'נכון'.
        """
        prompt += "---\nכעת, נתח את הטקסט הבא בהתאם לכללים לעיל.\n\n"
        prompt += f"""טקסט: {text}
    תשובה:"""

        return prompt


    def create_few_shot_prompt(self, text: str, label: str, n_examples: int) -> str:
        """
        Generates a few-shot learning prompt using examples.
        
        Args:
            text (str): The input legal text.
            label (str): The label to classify.
            n_examples (int): Number of few-shot examples to include.

        Returns:
            str: The formatted prompt.
        """

        label_description = self.label_descriptions.get(label, None)
        if label_description is None:
            print(f"Label '{label}' not found!")  # Debugging print
            return None

        examples = few_shot_examples.get(label, [])[:n_examples]

        prompt = f"""קבע אם הטקסט המשפטי הבא כולל את המידע הבא: {label_description}.
        התשובה יכולה להיות רק 'נכון' או 'לא נכון'.
        ענה אך ורק 'נכון' או 'לא נכון' ללא הסברים נוספים.
        
    הנחות:
    1. אם לא מצוין במפורש אדם אחר, יש להניח שההתייחסות היא לנאשם.
    2. אם הטקסט מתייחס לתיק אחר (למשל, "במקרה של ת"פ 111 הנאשם ירה..."), אין לתייג אותו כ'נכון'.
    3. אם מצוין שם של אדם אחר שאינו הנאשם (למשל, "צאלח ירה..."), אין לתייג את הטקסט כ'נכון'.
        דוגמאות:
        """

        for example in examples:
            prompt += f"""טקסט: {example['text']}
            תשובה: {'נכון' if example['label'] == 1 else 'לא נכון'}
            
            """
        prompt += "---\nכעת, נתח את הטקסט הבא בהתאם לכללים לעיל.\n\n"
        prompt += f"""טקסט: {text}
    תשובה:"""

        return prompt

    def create_cot_prompt(self, text: str, label: str) -> str:
        """
        Generates a chain-of-thought (CoT) prompt for reasoning-based classification.
        
        Args:
            text (str): The input legal text.
            label (str): The label to classify.

        Returns:
            str: The formatted prompt with step-by-step reasoning.
        """
        label_description = self.label_descriptions.get(label, None)
    
        if label_description is None:
            # If label description is not found, skip this label and continue
            print(f"Label '{label}' not found in label_descriptions. Skipping...")
            return None  # Or return a default prompt if needed, depending on your logic

        return f"""האם הטקסט המשפטי הבא מכיל מידע על {self.label_descriptions[label]}?
        
        טקסט: {text}
        
        נתח בקצרה:
        1. מהם האלמנטים המרכזיים בטקסט?
        2. האם הם קשורים ל-{self.label_descriptions[label]}?
        
        חשוב: ענה אך ורק 'נכון' או 'לא נכון' בהתבסס על הניתוח לעיל, ללא תוספת הסברים נוספים:
        תשובה:"""
    
    def create_cot_prompt_long(self, text: str, label: str) -> str:
        """
        Generates a detailed and expanded CoT prompt to analyze the given text step by step.
        """
        label_description = self.label_descriptions.get(label, None)
    
        if label_description is None:
            # If label description is not found, skip this label and continue
            print(f"Label '{label}' not found in label_descriptions. Skipping...")
            return None  # Or return a default prompt if needed, depending on your logic

        return f"""בוא ננתח שלב אחר שלב האם הטקסט המשפטי הבא מכיל מידע על {self.label_descriptions[label]}.
        
        טקסט: {text}
        
        שלבים לניתוח:
        1. מהם האלמנטים המרכזיים המוזכרים בטקסט? 
        2. האם האלמנטים שזוהו קשורים באופן ישיר או עקיף ל-{self.label_descriptions[label]}? 
        3. האם יש דיון מפורש או עקיף במשהו הקשור ל-{self.label_descriptions[label]}? 
        4. בהתבסס על האלמנטים והניתוח, האם ניתן להסיק שהטקסט מכיל מידע רלוונטי לנושא {self.label_descriptions[label]}?

        חשוב: ענה אך ורק 'נכון' או 'לא נכון' בהתבסס על הניתוח לעיל, ללא תוספת הסברים נוספים:
        תשובה:"""

    def get_prediction(self, prompt: str) -> bool:
        """
        Sends a classification request to OpenAI's model and retrieves the response.
        
        Args:
            prompt (str): The generated prompt for classification.
        
        Returns:
            bool: True if classified as relevant, False otherwise.
        """

        try:

            # Call the classifier and get the response
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=8,
                temperature=0
            )

            answer = response.choices[0].message.content.strip().lower()
            result= "נכון" in answer and "לא נכון" not in answer            
        ###  Log the prompt and result for debugging
            # print('------- PROMPT-------')
            # print(prompt)
            # print('------- RESPONSE-------')
            # print(response)
            # print(answer)
            # print('------- FINAL-------')
            # print(result)
            
            # Check if the result contains the desired logic
            return result
        except Exception as e:
            print(f"error in model: {e}")
            return False

    def evaluate_prompting_strategy(self,df: pd.DataFrame,label: str,prompt_type: str = 'zero-shot',n_examples: int = 3) -> Dict[str, float]:
        """
        Evaluates the classifier's performance using a specified prompting strategy.
        
        Args:
            df (pd.DataFrame): A DataFrame containing texts and labels.
            label (str): The label to classify.
            prompt_type (str, optional): The type of prompting ('zero-shot', 'few-shot', 'chain-of-thought'). Defaults to 'zero-shot'.
            n_examples (int, optional): Number of few-shot examples. Defaults to 3.
        
        Returns:
            dict: A dictionary containing accuracy, precision, recall, and F1-score.
        """
        texts = df['text'].tolist()
        true_labels = df[label].tolist()
        predictions = []
        results_data = []
        
        for text, true_label in tqdm(zip(texts, true_labels),desc=f"Processing {prompt_type} for {label}"):
            if prompt_type == 'zero-shot':
                prompt = self.create_zero_shot_prompt(text, label)
            elif prompt_type == 'few-shot':
                prompt = self.create_few_shot_prompt(text, label, n_examples)
            elif prompt_type == 'chain-of-thought':
                prompt = self.create_cot_prompt(text, label)
            else: # chain-of-thought-long
                prompt = self.create_cot_prompt_long(text, label)
                
            if not prompt:
                continue
   
            pred = self.get_prediction(prompt)
            generated_label = 1 if pred else 0
            predictions.append(generated_label)
            
            # Store result for export
            results_data.append({
                'text': text,
                'true_label': true_label,
                'generated_label': generated_label
            })
            time.sleep(1)  # Rate limiting
        
        # Save results to CSV
        results_df = pd.DataFrame(results_data)
        if prompt_type == 'few-shot':
            results_df.to_csv(f"{self.output_path}_wep_{label}_{prompt_type}_{n_examples}.csv", index=False)
        else:
            results_df.to_csv(f"{self.output_path}_wep_{label}_{prompt_type}.csv", index=False)
        
        precision, recall, f1, _ = precision_recall_fscore_support(
            true_labels, 
            predictions, 
            average='binary'
        )
        accuracy = accuracy_score(true_labels, predictions)
        
        return {
            'Aaccuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1_Score': f1
        }


### Drug classifier

In [ ]:
class HebrewLegalMultiLabelClassifierDrug:
    def __init__(self, output_path: str, model_name: str = "gpt-4o-mini"):
        self.client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        self.output_path = output_path
        self.model_name = model_name
        self.label_descriptions = {
             'RESPO':'האחריות על ביצוע העבירה',
              'REGRET':'הבעת חרטה',
             'CONFESSION': 'האם הנאשם הודה',
            'CIR_TYPE': ' סוג הסם המעורב בעבירה',
            'CIR_AMOUNT': ' ערכים מספריים עם יחידות מדידה (כגון גרם, ק"ג, טבליות) המדגישים את הכמות המדויקת של הסם.',
            'GENERAL_CIRCUM':
                "האם מופיע נסיבות כלליות להקלה על העונש?"
                "יש לתייג נכון גם אם מופיע אחד מהקריטריונים הבאים באופן משתמע, כל עוד ברור מההקשר שהם מתקיימים :"
                "(1) הפגיעה האפשרית של העונש בנאשם כולל פגיעה בלתי מידתית בנאשם בשל מאסר ארוך או נסיבותיו האישיות. "
                "(2) הפגיעה האפשרית של העונש במשפחתו של הנאשם; "
                "(3) הנזקים שנגרמו לנאשם מביצוע העבירה ומהרשעתו; "
                "(4) מאמצי הנאשם לשיקום ,שיקום עצמי, חזרה לחיים נורמטיביים , התרשמות שירות המבחן מהליך שיקום, שינוי חיובי,הרתעת ההליך הפלילי לנאשם"
                "(5) ניסיון הנאשם לתיקון תוצאות העבירה ולפיצוי הנפגעים; "
                "(6) שיתוף פעולה עם רשויות החוק, כולל אם ההליך הפלילי השפיע על הנאשם באופן מרתיע או משקם. "
                "(7) התנהגות חיובית של הנאשם ותרומתו לחברה "
                "(8)  נסיבות חייו של הנאשם טרם ביצוע העבירה, כולל השפעה של גיל, פחד, לחץ חברתי או רגשי על המעשים , מצוקה כלכלית או פגיעה אפשרית בפרנסה. בעיות רפואיות, נפשיות, או חוויות טראומטיות שהובילו לעבירה. השפעת החברה הסובבת, כולל התמכרויות ,הדרדרות לסמים, התדרדרות סביבתית. "
                "(9) התנהלות חריגה מצד רשויות החוק (עיכובים, עוולות וכו'). "
                "(10) חלוף זמן משמעותי מאז העבירה והשינוי שחל בנאשם מאז."
                "(11)  עברו הפלילי של הנאשם, כולל הרשעות קודמות"
                "אין לתייג נכון :אזכורים כלליים של חוקי ענישה,סיבות טכניות בלבד כמו חיסכון בזמן שיפוטי."
                "אין לתייג נכון אם מוזכר רק המלצת שירות המבחן ללא פירוט נוסף"
            ,
            'PUNISHMENT': "האם מופיע פירוט של פירוט מדויק של העונש שהוטל בהחלטה הנוכחית בלבד, כולל פרטים כמו סוג העונש (למשל, מאסר בפועל, מאסר על תנאי, קנס כספי, שלילת רישיון, צו מבחן, חילוט רכוש, השמדת סמים,  או שירות לתועלת הציבור) ומשך הזמן, תוך התמקדות בעונש הנוכחי בלבד ולא בהתייחסות לעונשים קודמים או להמלצות עתידיות.?",
            'CIR_ROLE':
            """
יש לתייג נכון אם אחד מהקריטריונים הבאים מתקיים, גם אם הוא משתמע מההקשר:
(1)	לנאשם היה תפקיד מרכזי בביצוע העבירה – יוזמה, תכנון, הובלה או מימון הפעולה.
(2)	הנאשם ביצע את העבירה כמו יבוא , הברחת סמים, גידול, אחזקה, או הפצה.
(3)	הנאשם שימש כמתווך – כלומר, סייע בהעברת הסם בין צדדים שונים או היה מעורב בהפצה.
(4)	הנאשם סייע באופן פעיל בביצוע העבירה – למשל, סיפק ציוד, מקום, או תמיכה לוגיסטית.
(5)	הנאשם שימש כבלדר או שליח להעברת סמים, בין אם ידע מראש את מטרת הפעולה ובין אם נוצל על ידי אחרים.
(6)	התיאור מבהיר את מעמדו של הנאשם בעבירה, למשל אם היה הדמות המרכזית או דמות שולית.
(7)	יש אזכור לתכנון, שיטתיות, או קישור לגורמים נוספים שמצביעים על מעורבות מהותית בעבירה.
(8)	הנאשם מכר או סיפק סם לאחרים, בין אם תמורת כסף ובין אם לא.
אין לתייג נכון אם:
הטקסט אינו מפרט את תפקידו של הנאשם או את רמת מעורבותו בעבירה.
(1)יש אזכור להחזקת סם בלבד ללא תיאור של ייבוא, מכירה, תיווך או הפצה.
(2)ההתייחסות היא לנסיבות חיצוניות כמו מצבו האישי של הנאשם, השלכות העבירה עליו, או רקע כללי ללא מידע על תפקידו בעבירה.
(3)מוזכר רק ההליך המשפטי או החוקי (למשל, התייחסות לעונש, הרשעה או פסיקה) ללא פירוט על תפקיד הנאשם.
(4)אין עדות לפעולה יזומה מצד הנאשם (למשל, אין אזכור למכירה, תיווך, גידול, ייבוא או הברחה).

            """
            ,
            'CIR_EQ': """
            מופיע אזכור למעבדה או לתשתית המשמשת לגידול, הפקה או הכנה של סמים. קיימת התייחסות ישירה למונח "מעבדה" או לתיאור ציוד המשמש לייצור סמים. ציוד כזה כולל אמצעים כגון גופי תאורה ייעודיים, מערכות אוורור והשקיה, אדניות, חומרי דישון, משקלים, שנאים, מפוחים, מסננים, נורות חימום, מזגנים, או כל רכיב טכני אחר שמטרתו לתמוך בתהליך הגידול והייצור של סמים.
בנוסף, זה חל גם על תיאורים של תכנון מוקדם להקמת מעבדה, שכירת מקום ייעודי לשם כך, רכישת ציוד בהיקף משמעותי, או ביצוע פעולות כמו חיבור פיראטי לחשמל לצורך תפעול המקום. כאשר מדובר במקרה שבו הוזכרה מעבדה במפורש, הדבר מהווה קריטריון חד-משמעי לסיווג לליבל , במקרים שבהם המונח "מעבדה" אינו מופיע, אך ישנו פירוט של ציוד מקצועי או מערכת מתקדמת לגידול והפקת סמים, עדיין ניתן לשייך את הקטע ללייבל זה.

            """
        }
        self.output_path = output_path

    def create_zero_shot_prompt(self, text: str, label: str) -> str:
        """
        Generates a zero-shot prompt for classification.

        Args:
            text (str): The input legal text.
            label (str): The label to classify.

        Returns:
            str: The formatted prompt.
        """

        label_description = self.label_descriptions.get(label, None)

        if label_description is None:
            # If label description is not found, skip this label and continue
            print(f"Label '{label}' not found in label_descriptions. Skipping...")
            return None  # Or return a default prompt if needed, depending on your logic

        prompt = f"""קבע אם הטקסט המשפטי כולל את המידע הבא: {label_description}.
        התשובה יכולה להיות רק 'נכון' או 'לא נכון'.
        ענה אך ורק 'נכון' או 'לא נכון' ללא הסברים נוספים.

    הנחות:
    1. אם לא מצוין במפורש אדם אחר, יש להניח שההתייחסות היא לנאשם.
    2. אם הטקסט מתייחס לתיק אחר (למשל, "במקרה של ע"פ 11/22 הנאשם ירה..."), אין לתייג אותו כ'נכון' באף מקרה.
    3. אם מצוין שם של אדם אחר שאינו הנאשם (למשל, "פלוני ירה..."), אין לתייג את הטקסט כ'נכון'.
        """
        prompt += "---\nכעת, נתח את הטקסט הבא בהתאם לכללים לעיל.\n\n"
        prompt += f"""טקסט: {text}
    תשובה:"""

        return prompt
    def create_few_shot_prompt(self, text: str, label: str, n_examples: int) -> str:
        """
        Generates a few-shot learning prompt using examples.

        Args:
            text (str): The input legal text.
            label (str): The label to classify.
            n_examples (int): Number of few-shot examples to include.

        Returns:
            str: The formatted prompt.
        """

        label_description = self.label_descriptions.get(label, None)
        if label_description is None:
            print(f"Label '{label}' not found!")  # Debugging print
            return None

        examples = few_shot_examples.get(label, [])[:n_examples]

        prompt = f"""קבע אם הטקסט המשפטי הבא כולל את המידע הבא: {label_description}.
        התשובה יכולה להיות רק 'נכון' או 'לא נכון'.
        ענה אך ורק 'נכון' או 'לא נכון' ללא הסברים נוספים.

    הנחות:
    1. אם לא מצוין במפורש אדם אחר, יש להניח שההתייחסות היא לנאשם.
    2. אם הטקסט מתייחס לתיק אחר (למשל, "במקרה של ת"פ 111 הנאשם ירה..."), אין לתייג אותו כ'נכון'.
    3. אם מצוין שם של אדם אחר שאינו הנאשם (למשל, "צאלח ירה..."), אין לתייג את הטקסט כ'נכון'.
        דוגמאות:
        """

        for example in examples:
            prompt += f"""טקסט: {example['text']}
            תשובה: {'נכון' if example['label'] == 1 else 'לא נכון'}

            """
        prompt += "---\nכעת, נתח את הטקסט הבא בהתאם לכללים לעיל.\n\n"
        prompt += f"""טקסט: {text}
    תשובה:"""

        return prompt
    def create_cot_prompt(self, text: str, label: str) -> str:
        """
        Generates a concise CoT prompt to analyze the given text with minimal reasoning.
        """
        label_description = self.label_descriptions.get(label, None)

        if label_description is None:
            # If label description is not found, skip this label and continue
            print(f"Label '{label}' not found in label_descriptions. Skipping...")
            return None  # Or return a default prompt if needed, depending on your logic

        return f"""האם הטקסט המשפטי הבא מכיל מידע על {self.label_descriptions[label]}?

        טקסט: {text}

        נתח בקצרה:
        1. מהם האלמנטים המרכזיים בטקסט?
        2. האם הם קשורים ל-{self.label_descriptions[label]}?

        חשוב: ענה אך ורק 'נכון' או 'לא נכון' בהתבסס על הניתוח לעיל, ללא תוספת הסברים נוספים:
        תשובה:"""

    def create_cot_prompt_long(self, text: str, label: str) -> str:
        """
        Generates a detailed and expanded CoT prompt to analyze the given text step by step.
        """
        label_description = self.label_descriptions.get(label, None)

        if label_description is None:
            # If label description is not found, skip this label and continue
            print(f"Label '{label}' not found in label_descriptions. Skipping...")
            return None  # Or return a default prompt if needed, depending on your logic

        return f"""בוא ננתח שלב אחר שלב האם הטקסט המשפטי הבא מכיל מידע על {self.label_descriptions[label]}.

        טקסט: {text}

        שלבים לניתוח:
        1. מהם האלמנטים המרכזיים המוזכרים בטקסט?
        2. האם האלמנטים שזוהו קשורים באופן ישיר או עקיף ל-{self.label_descriptions[label]}?
        3. האם יש דיון מפורש או עקיף במשהו הקשור ל-{self.label_descriptions[label]}?
        4. בהתבסס על האלמנטים והניתוח, האם ניתן להסיק שהטקסט מכיל מידע רלוונטי לנושא {self.label_descriptions[label]}?

        חשוב: ענה אך ורק 'נכון' או 'לא נכון' בהתבסס על הניתוח לעיל, ללא תוספת הסברים נוספים:
        תשובה:"""

    def get_prediction(self, prompt: str) -> bool:
        try:
            # Call the classifier and get the response
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=5,
                temperature=0
            )

            answer = response.choices[0].message.content.strip().lower()
            result= "נכון" in answer and "לא נכון" not in answer
            return result
        except Exception as e:
            print(f"error in model: {e}")
            return False


    def evaluate_prompting_strategy(
        self,
        df: pd.DataFrame,
        label: str,
        prompt_type: str = 'zero-shot',
        n_examples: int = 3
    ) -> Dict[str, float]:
        texts = df['text'].tolist()
        true_labels = df[label].tolist()
        predictions = []
        results_data = []



        for text, true_label in tqdm(zip(texts, true_labels),desc=f"Processing {prompt_type} for {label}"):
            if prompt_type == 'zero-shot':
                prompt = self.create_zero_shot_prompt(text, label)
            elif prompt_type == 'few-shot':
                prompt = self.create_few_shot_prompt(text, label, n_examples)
            elif prompt_type == 'chain-of-thought':
                prompt = self.create_cot_prompt(text, label)
            else: # chain-of-thought-long
                prompt = self.create_cot_prompt_long(text, label)

            pred = self.get_prediction(prompt)
            generated_label = 1 if pred else 0
            predictions.append(generated_label)

            # Store result for export
            results_data.append({
                'text': text,
                'true_label': true_label,
                'generated_label': generated_label
            })
            time.sleep(1)  # Rate limiting

        # Save results to CSV
        results_df = pd.DataFrame(results_data)
        if prompt_type == 'few-shot':
            results_df.to_csv(f"{self.output_path}_drug_{label}_{prompt_type}_{n_examples}.csv", index=False)
        else:
            results_df.to_csv(f"{self.output_path}_drug_{label}_{prompt_type}.csv", index=False)

        precision, recall, f1, _ = precision_recall_fscore_support(
            true_labels,
            predictions,
            average='binary'
        )
        accuracy = accuracy_score(true_labels, predictions)

        return {
            'Aaccuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1_Score': f1
        }


### Setup

In [ ]:
os.environ["OPENAI_API_KEY"] = "api-key" 
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

data_file = "/Data/tagged_data_manualy/weapon/test.csv"
output_directory = "/Data/tagged_data_manualy/weapon/results/" 
Drug_or_Wep = "Wep"  # Change here wich classifier to use; Drug or Wep
example_counts = [3] # Number of examples per label to use in the few-shot experiments
labels=["CONFESSION","CIR_TYPE_WEP","CIR_HELD_WAY_WEP","CIR_AMMU_AMOUNT_WEP","CIR_PURPOSE","GENERAL_CIRCUM","CIR_STATUS_WEP","REGRET","PUNISHMENT","CIR_PLANNING","RESPO","CIR_OBTAIN_WAY_WEP","CIR_USE"]
CSV_LOG_FILE = "batch_log.csv"


## Sequential processing

### Utils

In [ ]:
def run_multilabel_experiments(data_file: str,classifier=HebrewLegalMultiLabelClassifierWep(""),labels_to_test: List[str] = None,n_examples: int = 3,prompt_types: List[str] = ['zero-shot', 'few-shot', 'chain-of-thought','chain-of-thought-long']) -> Dict[str, Dict[str, Dict[str, float]]]:
    """
    Runs multi-label classification experiments on legal text data using different prompting strategies.

    Parameters:
    -----------
    data_file : str
        Path to the CSV file containing the dataset.
    classifier : HebrewLegalMultiLabelClassifierWep or HebrewLegalMultiLabelClassifierDrug
        The classifier instance responsible for evaluating the text classification.
    labels_to_test : List[str], optional
        List of labels (column names) to evaluate. If None, all suitable labels from the dataset will be used.
    n_examples : int, default=3
        Number of few-shot examples to include in the prompt.
    prompt_types : List[str], default=['zero-shot', 'few-shot', 'chain-of-thought', 'chain-of-thought-long']
        List of different prompt types to use for classification.

    Returns:
    --------
    results : Dict[str, Dict[str, Dict[str, float]]]
        A nested dictionary containing classification results per label
    """   
    results = {}
    
    # Read the dataset
    df = pd.read_csv(data_file)
    # Keep only relevant columns for text and label
    if labels_to_test is not None:
        df = df[['text'] + labels_to_test]  # Keep 'text' and the labels to test

    # Drop rows where 'text' or any label in 'labels_to_test' is NaN
    df = df.dropna(subset=['text'] + labels_to_test)

    print(df.head())
    # If no labels are specified, default to all columns except 'text'
    if labels_to_test is None:
        labels_to_test = [col for col in df.columns if col != 'text' and df[col].dtype in ['int64', 'bool']]
        
    print(labels_to_test)
    
    for label in labels_to_test:
        if label == 'reject' or label == 'verdict':
            continue
        print(f"\nProcessing label: {label}")
        label_results = {}
        
        for prompt_type in prompt_types:
            print(f"Running evaluation for {prompt_type} with {n_examples} examples...")
            # generate_jsonl_file(df, label,prompt_type,n_examples,classifier)
            label_results[f"{prompt_type} - {n_examples} examples"] = classifier.evaluate_prompting_strategy(
                df,
                label,
                prompt_type,
                n_examples=n_examples
            )

        results[label] = label_results
    
    return results


### Main


In [ ]:
if __name__ == "__main__":
    """
    Main execution script for running multi-label classification experiments.

    Steps:
    1. Creates the output directory if it does not exist.
    2. Initializes the classifier based on the category (weapons or drugs).
    3. Iterates over different numbers of few-shot examples.
    4. Sets the appropriate prompt types for each experiment.
    5. Runs the classification experiments for each label.
    6. Prints the classification results, including performance metrics.
    """

    
    os.makedirs(output_directory, exist_ok=True)
    
    if Drug_or_Wep == "Wep":
        classifier = HebrewLegalMultiLabelClassifierWep(output_path=output_directory)
    else:
        classifier = HebrewLegalMultiLabelClassifierDrug(output_path=output_directory)
    
    
    # Iterate over the number of examples and run experiments
    for n_samples in example_counts:
        print(f"\nRunning experiments with {n_samples} examples per label...\n")
        
        # Set prompt types conditionally
        if n_samples == 1:
            prompt_types = ['zero-shot', 'few-shot', 'chain-of-thought', 'chain-of-thought-long']
        else:
            prompt_types = ['few-shot']
        
        # Run the experiment with the specified prompt types
        results = run_multilabel_experiments(
            data_file=data_file,
            n_examples=n_samples,
            prompt_types=prompt_types,  # Pass the prompt types
            classifier = classifier,
            labels_to_test=labels
            
        )
        
        # Print results
        print(f"Results for {n_samples} examples per label:")
        for label, strategies in results.items():
            print(f"\nResults for label: {label}")
            for strategy, metrics in strategies.items():
                print(f"\nStrategy: {strategy}")
                for metric, value in metrics.items():
                    print(f"{metric}: {value:.4f}")


## batch processing

### Utils

In [ ]:
# Function to create batch file
def generate_jsonl_file(output_directory,df, label,prompt_type,n_examples,classifier,Drug_or_Wep):
    """
    Generates a JSONL batch file for classification requests and saves metadata for tracking.

    Parameters:
    -----------
    output_directory : str
        Path to the directory where batch files will be saved.
    df : pd.DataFrame
        Dataframe containing the input text and labels.
    label : str
        The label being classified.
    prompt_type : str
        The type of prompting strategy (e.g., 'zero-shot', 'few-shot', 'chain-of-thought').
    n_examples : int
        Number of few-shot examples to include in the prompt.
    classifier : object
        The classifier instance used to generate prompts.
    Drug_or_Wep : str
        Indicates whether the classification is for "drugs" or "weapons".

    Returns:
    --------
    str
        The path to the batch file directory.
    """
    batch_file_dir= os.path.join(output_directory,f"batch_files_{Drug_or_Wep}")
    os.makedirs(batch_file_dir, exist_ok=True)
    batch_file_path = os.path.join(batch_file_dir,f"batch_{label}_{prompt_type}.jsonl")
    metadata_file_path = batch_file_path.replace(".jsonl", "_metadata.json")

    texts = df['text'].tolist()
    true_labels = df[label].tolist()
    metadata = {}  # Store mapping of custom_id → (text, true_label)

    with open(batch_file_path, "w", encoding="utf-8") as f:
        for text, true_label in tqdm(zip(texts, true_labels),desc=f"Processing {prompt_type} for {label}"):
            if prompt_type == 'zero-shot':
                prompt = classifier.create_zero_shot_prompt(text, label)
            elif prompt_type == 'few-shot':
                prompt = classifier.create_few_shot_prompt(text, label, n_examples)
            elif prompt_type == 'chain-of-thought':
                prompt = classifier.create_cot_prompt(text, label)
            else: # chain-of-thought-long
                prompt = classifier.create_cot_prompt_long(text, label)
                
            if not prompt:
                continue

            # Unique ID for tracking responses
            custom_id = str(uuid.uuid4())

            # Store metadata in a separate file
            metadata[custom_id] = {"text": text, "true_label": true_label}
            if label=="GENERAL_CIRCUM" :
                request = {
                "custom_id": custom_id,
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "messages": [{"role": "user", "content": prompt}],
                    "model": "gpt-4o",
                    "max_tokens": 8,
                    "temperature": 0.0
                    }
            }
            else:
                request = {
                "custom_id": custom_id,
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "messages": [{"role": "user", "content": prompt}],
                    "model": "gpt-4o-mini",
                    "max_tokens": 8,
                    "temperature": 0.0
                }
            }
            f.write(json.dumps(request) + "\n")

    # Save metadata separately
    with open(metadata_file_path, "w", encoding="utf-8") as meta_f:
        json.dump(metadata, meta_f, ensure_ascii=False, indent=4)

    return batch_file_dir

def run_multilabel_experiments_batches(output_directory:str,Drug_or_Wep:str,data_file: str,classifier=HebrewLegalMultiLabelClassifierWep(""),labels_to_test: List[str] = None,n_examples: int = 3,prompt_types: List[str] = ['zero-shot', 'few-shot', 'chain-of-thought','chain-of-thought-long']) -> Dict[str, Dict[str, Dict[str, float]]]:
    """
    Runs batch classification experiments and generates batch files for each label and prompt type.
    """

    # Read the dataset
    df = pd.read_csv(data_file)
    # Keep only relevant columns for text and label

    if labels_to_test is not None:
        df = df[['text'] + labels_to_test]  # Keep 'text' and the labels to test

    # Drop rows where 'text' or any label in 'labels_to_test' is NaN
    df = df.dropna(subset=['text'] + labels_to_test)
    df = df.drop_duplicates()

    print(df.head())
    # If no labels are specified, default to all columns except 'text'
    if labels_to_test is None:
        labels_to_test = [col for col in df.columns if col != 'text' and df[col].dtype in ['int64', 'bool']]
        
    print(labels_to_test)
    
    for label in labels_to_test:
        if label == 'reject' or label == 'verdict':
            continue
        print(f"\nProcessing label: {label}")
        
        for prompt_type in prompt_types:
            print(f"genrate batch file for {prompt_type} with {n_examples} examples...")
            batch_file_dir=generate_jsonl_file(output_directory,df, label,prompt_type,n_examples,classifier,Drug_or_Wep)
    return batch_file_dir

In [ ]:

# Function to upload a file to OpenAI
def upload_file(client,file_path):
    """
    Uploads a file to OpenAI for batch processing.
    """
    with open(file_path, "rb") as f:
        response = client.files.create(
            file=f,
            purpose="batch"
        )
    return response.id  # Returns the uploaded file ID

# Function to submit batch files using uploaded file ID
def submit_batch(client,input_file_id):
    """
    Submits a batch job to OpenAI.
    """

    try:
        response = client.batches.create(
            input_file_id=input_file_id,  # Correct parameter name
            endpoint="/v1/chat/completions",
            completion_window= "24h"
        )
        print(f"Batch submitted successfully! Batch ID: {response.id}")
        return response.id  # Returns the batch job ID
    except openai.OpenAIError as e:
        print(f"Error submitting batch: {e}")
        return None


def download_batch_results(client, batch_status, batch_file, batch_dir):
    """
    Downloads batch results from OpenAI.

    Parameters:
    -----------
    client : OpenAI client object
        The API client for OpenAI.
    batch_status : object
        The status object of the batch job.
    batch_file : str
        The batch file name.
    batch_dir : str
        Directory where results will be stored.

    Returns:
    --------
    str
        Path to the saved results file.
    pd.DataFrame
        DataFrame containing processed batch results.
    """

    if batch_status.status == "completed":
        output_file_path = os.path.join(batch_dir, f"results_{batch_file.replace('.jsonl', '.json')}")
        output_csv_path = os.path.join(batch_dir, f"results_{batch_file.replace('.jsonl', '.csv')}")
        metadata_file_path = os.path.join(batch_dir, f"{batch_file.replace('.jsonl', '_metadata.json')}")

        # Retrieve file content
        response = client.files.content(batch_status.output_file_id)

        # Read content properly
        file_content = response.read()  # Ensure we get bytes

        with open(output_file_path, "wb") as f:
            f.write(file_content)  # Write the binary content
        
        print(f"Results saved to: {output_file_path}")

        df_results = process_batch_results(output_file_path, metadata_file_path,output_csv_path)


        return output_file_path,df_results
    return None



def process_batch_results(results_file, metadata_file, output_csv):
    """
    Processes batch results and saves them as a CSV file.
    """
    # Check if metadata file exists
    if not os.path.exists(metadata_file):
        print(f"Metadata file not found: {metadata_file}")
        return None

    # Load metadata
    with open(metadata_file, "r", encoding="utf-8") as meta_f:
        try:
            metadata = json.load(meta_f)
        except json.JSONDecodeError:
            print("Error: Metadata file is not valid JSON.")
            return None

    # Load results and add GPT response to each record
    results = []
    with open(results_file, "r", encoding="utf-8") as f:
        for line in f:
            try:
                response_data = json.loads(line)
                custom_id = response_data.get("custom_id")
                answer = response_data.get("response", {})\
                                      .get("body", {})\
                                      .get("choices", [{}])[0]\
                                      .get("message", {})\
                                      .get("content", "").strip()

                # Compute generated label: "נכון" must be present and "לא נכון" must NOT be present
                generated_label_ = "נכון" in answer and "לא נכון" not in answer
                generated_label = 1 if generated_label_ else 0

                # Match with metadata
                if custom_id in metadata:
                    text = metadata[custom_id]["text"]
                    true_label = metadata[custom_id]["true_label"]
                    # Append full GPT response as well as computed labels
                    results.append({
                        "text": text,
                        "true_label": true_label,
                        "generated_label": generated_label,
                        "gpt_response": answer
                    })
            except json.JSONDecodeError:
                print(f"Skipping malformed JSON line: {line}")

    if not results:
        print("No valid results found in batch output.")
        return None

    # Save results to CSV
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False, encoding="utf-8")
    print(f"CSV saved to: {output_csv}")

    return df

def log_batch_completion(batch_file, batch_id,CSV_LOG_FILE):
    """Logs completed batches in a CSV file."""
    file_exists = os.path.exists(CSV_LOG_FILE)
    with open(CSV_LOG_FILE, mode='a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(["batch_file", "batch_id"])  # Write header if file doesn't exist
        writer.writerow([batch_file, batch_id])



### Main

In [ ]:
if __name__ == "__main__":
    """
    Main execution script for batch processing:
    1. Creates the output directory if it doesn't exist.
    2. Initializes the classifier based on the category (weapons or drugs).
    3. Generates batch files for each label and prompt type.
    4. Uploads the batch files to OpenAI for processing.
    5. Submits the uploaded files as batch jobs.
    6. Logs submitted batch files.
    """

    os.makedirs(output_directory, exist_ok=True)
    
    if Drug_or_Wep == "Wep":
        classifier = HebrewLegalMultiLabelClassifierWep(output_path=output_directory)
    else:
        classifier = HebrewLegalMultiLabelClassifierDrug(output_path=output_directory)

    # Step 1: Create all batch files
    for n_samples in example_counts:
        print(f"\n🔹 Running experiments with {n_samples} examples per label...\n")
        
        prompt_types = ['few-shot'] if n_samples != 1 else ['zero-shot', 'few-shot', 'chain-of-thought', 'chain-of-thought-long']
        # prompt_types=['zero-shot']
        batch_dir = run_multilabel_experiments_batches(
            output_directory=output_directory,
            data_file=data_file,
            n_examples=n_samples,
            prompt_types=prompt_types,  
            classifier=classifier,
            labels_to_test=labels,
            Drug_or_Wep=Drug_or_Wep
        )
    
    # Step 2: List batch files for each label
    batch_files = []
    for label in labels:
        label_batch_dir = os.path.join(output_directory, f"batch_files_{Drug_or_Wep}")
        label_files = [f for f in os.listdir(label_batch_dir) if f.startswith(f"batch_{label}_") and f.endswith(".jsonl")]
        batch_files.extend(label_files)

    batch_jobs = {}
    start_time = time.time()
    
    for batch_file in batch_files:
        batch_path = os.path.join(batch_dir, batch_file)
        file_id = upload_file(client, batch_path)
        print(f"✅ Uploaded {batch_file}, File ID: {file_id}")
        
        batch_id = submit_batch(client, file_id)
        batch_jobs[batch_file] = batch_id
        print(f"🚀 Submitted {batch_file} with Batch ID: {batch_id}")
        
        # Start a new batch every 30 sec
        while time.time() - start_time < 30:
            time.sleep(5)  # Small wait to avoid busy looping
        start_time = time.time()



Monitor batches

In [ ]:
"""
    Monitors the status of submitted batch jobs and handles retries for failed or cancelled jobs.
    
    Steps:
    1. Retrieve the status of each active batch from OpenAI.
    2. If a batch is completed, log the completion and download the results.
    3. If a batch fails, re-upload the corresponding file and resubmit the batch.
    4. If a batch is cancelled, re-upload and resubmit the batch.
    5. If a batch is still in progress, print the current status, including total, completed, and failed requests.
    6. Attempt to retrieve token usage statistics if available.
    7. Wait for 2 minutes before checking batch statuses again.
"""
while batch_jobs:
    for batch_file, batch_id in list(batch_jobs.items()):
        batch_status = client.batches.retrieve(batch_id)
        if batch_status.status == "completed":
            print(f"Batch {batch_id} ({batch_file}) completed!")
            log_batch_completion(batch_file, batch_id,CSV_LOG_FILE)
            download_batch_results(client, batch_status, batch_file, batch_dir)
            del batch_jobs[batch_file]  # Remove completed batch from tracking
        elif batch_status.status == "failed":
            print(f" Batch {batch_id} ({batch_file}) failed! Restarting...")
            file_id = upload_file(client, os.path.join(batch_dir, batch_file))
            new_batch_id = submit_batch(client, file_id)
            batch_jobs[batch_file] = new_batch_id  # Replace failed batch with new one
            print(f" Restarted {batch_file} with new Batch ID: {new_batch_id}")
        elif batch_status.status == "cancelled":
            print(f"⚠️ Batch {batch_id} ({batch_file}) was cancelled! Restarting...")
            file_id = upload_file(client, os.path.join(batch_dir, batch_file))
            new_batch_id = submit_batch(client, file_id)
            batch_jobs[batch_file] = new_batch_id
            print(f"Restarted {batch_file} with new Batch ID: {new_batch_id}")
        elif batch_status.status == "in_progress":
            # Print batch status and request counts
            print(f"Batch ID: {batch_status.id}")
            print(f"Batch file: {batch_file}")
            print(f"Status: {batch_status.status}")
            print(f"Total Requests: {batch_status.request_counts.total}")
            print(f"Completed Requests: {batch_status.request_counts.completed}")
            print(f"Failed Requests: {batch_status.request_counts.failed}")
            print("__________________________")

    print(f" Checking batch statuses again in 2 minutes...")
    time.sleep(120)

print("\n All batches have been processed successfully!")


### Evaluations

In [ ]:
def evaluate_results(output_directory, labels, Drug_or_Wep):
    """
    Evaluates model performance for each label based on classification metrics.

    Parameters:
    -----------
    output_directory : str
        Path to the output directory where batch results are stored.
    labels : list
        List of classification labels to evaluate.
    Drug_or_Wep : str
        Specifies if the evaluation is for "drugs" or "weapons".

    Returns:
    --------
    pd.DataFrame
        A DataFrame containing precision, recall, F1-score, and accuracy for each label.

    Functionality:
    --------------
    1. Reads the result CSV for each label.
    2. Ensures labels contain only binary values (0 or 1).
    3. Computes precision, recall, F1-score, and accuracy.
    4. Prints and returns the evaluation results as a DataFrame.
    """

    results = []

    for label in labels:
        csv_path = os.path.join(output_directory, f"batch_files_{Drug_or_Wep}", f"results_batch_{label}_few-shot.csv")
        
        if os.path.exists(csv_path):
            df = pd.read_csv(csv_path)

            # Ensure labels are binary (0/1)
            if not set(df['true_label'].unique()).issubset({0,1}) or not set(df['generated_label'].unique()).issubset({0,1}):
                print(f"Skipping {label}: Labels are not binary!")
                continue
            
            precision = precision_score(df['true_label'], df['generated_label'], average='binary', pos_label=1)
            recall = recall_score(df['true_label'], df['generated_label'], average='binary', pos_label=1)
            f1 = f1_score(df['true_label'], df['generated_label'], average='binary', pos_label=1)
            accuracy = accuracy_score(df['true_label'], df['generated_label'])
            
            results.append({
                'Label': label,
                'Precision': precision,
                'Recall': recall,
                'F1-Score': f1,
                'Accuracy': accuracy
            })
        else:
            print(f"File not found: {csv_path}")
    
    results_df = pd.DataFrame(results)
    print(results_df)
    return results_df

evaluate_results(output_directory, labels, Drug_or_Wep)
